<a href="https://colab.research.google.com/github/williammendat/DataScience-Pojekt/blob/main/data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A first look at the data

In [ ]:
#get data from server
!wget --no-check-certificate https://www.keuper-labs.org/project_ss22.zip

--2022-05-22 19:04:54--  https://www.keuper-labs.org/project_ss22.zip
Resolving www.keuper-labs.org (www.keuper-labs.org)... 91.250.118.129
Connecting to www.keuper-labs.org (www.keuper-labs.org)|91.250.118.129|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 70859177 (68M) [application/zip]
Saving to: ‘project_ss22.zip’

project_ss22.zip    100%[===================>]  67.58M  11.4MB/s    in 6.5s    

2022-05-22 19:05:01 (10.4 MB/s) - ‘project_ss22.zip’ saved [70859177/70859177]



In [ ]:
#unpack
!unzip project_ss22.zip

Archive:  project_ss22.zip
   creating: dressipi_recsys2022/
  inflating: dressipi_recsys2022/candidate_items.csv  
  inflating: dressipi_recsys2022/item_features.csv  
  inflating: dressipi_recsys2022/test_leaderboard_sessions.csv  
  inflating: dressipi_recsys2022/test_final_sessions.csv  
  inflating: dressipi_recsys2022/README.txt  
  inflating: dressipi_recsys2022/train_purchases.csv  
  inflating: dressipi_recsys2022/train_sessions.csv  
  inflating: dressipi_recsys2022/README_win.txt  


In [ ]:
import pandas as pd

Get the necessary csv files

In [ ]:
items = pd.read_csv('dressipi_recsys2022/candidate_items.csv')
features = pd.read_csv('dressipi_recsys2022/item_features.csv')
train_purchases = pd.read_csv('dressipi_recsys2022/train_purchases.csv')
train_sessions = pd.read_csv('dressipi_recsys2022/train_sessions.csv')
test_leaderboard_sessions = pd.read_csv('dressipi_recsys2022/test_leaderboard_sessions.csv')

## have a first look

In [ ]:
test_leaderboard_sessions.head(10)

,session_id,item_id,date
0,26,19185,2021-06-16 09:53:54.158
1,200,17089,2021-06-25 12:23:40.811
2,200,17089,2021-06-25 12:24:36.631
3,200,8060,2021-06-25 12:24:41.677
4,200,4758,2021-06-25 12:24:50.692
5,205,8194,2021-06-11 00:28:07.058
6,495,6853,2021-06-14 22:13:06.741
7,521,26471,2021-06-19 13:50:03.09
8,587,22204,2021-06-01 16:43:22.80
9,721,13914,2021-06-19 18:46:57.263


In [ ]:
#how many items do we have?
items.describe()

,item_id
count,4990.000000
mean,14007.035271
std,8218.231425
min,4.000000
25%,6833.500000
50%,14108.500000
75%,21200.000000
max,28137.000000


In [ ]:
features.head(5)

,item_id,feature_category_id,feature_value_id
0,2,56,365
1,2,62,801
2,2,68,351
3,2,33,802
4,2,72,75


In [ ]:
features.iloc[[135]]

,item_id,feature_category_id,feature_value_id
135,10,72,75


In [ ]:
features.loc[features["item_id"]== 10]

,item_id,feature_category_id,feature_value_id
125,10,56,365
126,10,15,451
127,10,63,816
128,10,12,410
129,10,30,564
130,10,61,706
131,10,50,128
132,10,69,639
133,10,7,798
134,10,30,482


Get the items with duplicated category ids 

In [ ]:
gp = features.groupby(["item_id", "feature_category_id"])
df = gp.count()
df[df["feature_value_id"] != 1]

,,feature_value_id
item_id,feature_category_id,
10,30,2
11,30,3
18,30,4
30,4,3
35,30,2
...,...,...
28125,30,3
28126,30,3
28133,28,2


In [ ]:
features["item_id"].describe()

count    471751.000000
mean      14058.539477
std        8107.465455
min           2.000000
25%        7060.000000
50%       14045.000000
75%       21063.000000
max       28143.000000
Name: item_id, dtype: float64

In [ ]:
features["feature_category_id"].max()

73

**Base Line algo **

In [ ]:
import random

def create_item_suggestions(df_session):
  df_current = pd.DataFrame(columns=['session_id', 'item_id', 'rank'])
  ser_i = pd.Series({'session_id':df_session.iloc[[-1]]['session_id'].values[0],
                    'item_id': df_session.iloc[[-1]]['item_id'].values[0],
                    'rank':1})
  df_current = df_current.append(ser_i, ignore_index=True)
  for i in range(2, 100):
    rn = random.randint(0, 4490)
    ser_i = pd.Series({'session_id':df_session.iloc[[-1]]['session_id'].values[0],
                    'item_id': items.iloc[[rn]]['item_id'].values[0],
                    'rank':i})
    df_current = df_current.append(ser_i, ignore_index=True)
    
  return df_current

In [ ]:
def create_submission_file(file_name, df_sessions):
  df_all = pd.DataFrame(columns=['session_id', 'item_id', 'rank'])
  df_ses = df_sessions.groupby("session_id")
  for sess in df_ses.indices:
    df = create_item_suggestions(df_ses.get_group(sess))
    df_all = df_all.append(df)
    
  df_all.to_csv(file_name, index=False)

In [ ]:
create_submission_file("test_df.csv", test_leaderboard_sessions)

**Feature Extraction**

Get one DataFrame with itemid the feature category id (as list) and the feature value id (as list)

In [ ]:
itemGroupedDataframe=features.groupby("item_id", as_index=False).agg({'feature_category_id':lambda x: list(x), 'feature_value_id':lambda x: list(x)})
itemGroupedDataframe

,item_id,feature_category_id,feature_value_id
0,2,"[56, 62, 68, 33, 72, 29, 16, 50, 61, 53, 7, 69...","[365, 801, 351, 802, 75, 123, 38, 76, 462, 6, ..."
1,3,"[56, 69, 68, 17, 32, 11, 45, 7, 19, 46, 61, 73...","[365, 592, 14, 378, 902, 859, 559, 452, 254, 8..."
2,4,"[55, 17, 5, 69, 18, 68, 50, 73, 65, 7, 59, 47,...","[267, 378, 605, 538, 289, 373, 317, 544, 521, ..."
3,7,"[56, 50, 63, 41, 26, 32, 61, 68, 69, 7, 15, 44...","[153, 708, 816, 759, 268, 902, 706, 739, 592, ..."
4,8,"[56, 55, 7, 69, 72, 3, 45, 18, 59, 73, 5, 61, ...","[365, 267, 798, 592, 75, 793, 559, 817, 856, 5..."
...,...,...,...
23686,28139,"[73, 50, 68, 3, 41, 17, 45, 7, 44, 46, 26, 5, ...","[544, 901, 106, 793, 145, 378, 559, 798, 621, ..."
23687,28140,"[37, 67, 23, 39, 2, 47, 56, 61, 36, 31, 64, 68...","[469, 349, 510, 731, 53, 720, 365, 706, 678, 7..."
23688,28141,"[56, 5, 48, 72, 4, 68, 46, 3, 73, 17, 47, 26, ...","[365, 605, 354, 75, 719, 379, 825, 889, 544, 3..."
23689,28142,"[56, 20, 7, 73, 32, 65, 69, 68, 19, 72, 26, 41...","[153, 342, 619, 91, 522, 610, 740, 895, 232, 7..."


In [ ]:
itemGroupedDataframe.columns

Index(['item_id', 'feature_category_id', 'feature_value_id'], dtype='object')

Check if we can interate to all columns

Create a item class with item id and a dict of category id and value

In [ ]:
class Item:
  id = 0
  categorydict = dict()
  
  def __init__(self, id, category_count):
    self.id = id
    for i in range(1, category_count + 1):
      self.categorydict[i] = 0;

  def addToDict(self, key, value):
    self.categorydict[key] = self.categorydict.get(key, 0) + value
  

Get the category count

In [ ]:
categoryCount = features["feature_category_id"].max()
categoryCount

73

Test Code to validate the class idee

In [ ]:
firstRow = itemGroupedDataframe.iloc[0]
firstRow

item_id                                                                2
feature_category_id    [56, 62, 68, 33, 72, 29, 16, 50, 61, 53, 7, 69...
feature_value_id       [365, 801, 351, 802, 75, 123, 38, 76, 462, 6, ...
Name: 0, dtype: object

In [ ]:
secoundRow = itemGroupedDataframe.iloc[2]
secoundRow

item_id                                                                4
feature_category_id    [55, 17, 5, 69, 18, 68, 50, 73, 65, 7, 59, 47,...
feature_value_id       [267, 378, 605, 538, 289, 373, 317, 544, 521, ...
Name: 2, dtype: object

In [ ]:
len(firstRow['feature_category_id'])

13

In [ ]:
len(firstRow['feature_value_id'])

13

Create a list with every item

In [ ]:
itemDict = dict()

for index, row in itemGroupedDataframe.iterrows():
  itemClass = Item(row['item_id'], categoryCount)

  for i in range(0, len(row['feature_category_id'])):
    itemClass.addToDict(row['feature_category_id'][i], row['feature_value_id'][i])

  itemDict[itemClass.id] = list(itemClass.categorydict.values()).copy()

Vorbereitung der test daten 


In [ ]:
train_session_sessionId_group_dataframe = train_sessions.groupby("session_id", as_index=False).agg({'item_id':lambda x: list(x), 'date':lambda x: list(x)})
train_session_sessionId_group_dataframe

,session_id,item_id,date
0,3,"[9655, 9655]","[2020-12-18 21:25:00.373, 2020-12-18 21:19:48...."
1,13,[15654],[2020-03-13 19:35:27.136]
2,18,"[18316, 2507, 4026]","[2020-08-26 19:18:30.833, 2020-08-26 19:16:31...."
3,19,"[25772, 6341, 25555, 20033, 8281, 8268, 4385, ...","[2020-11-02 16:31:18.543, 2020-11-02 16:34:33...."
4,24,"[2927, 11662, 2927, 28075, 434, 16064, 10414, ...","[2020-02-26 17:23:04.84, 2020-02-26 17:26:29.1..."
...,...,...,...
999995,4439986,"[13914, 19310, 13403, 23502, 27733, 12373]","[2021-05-13 11:48:21.792, 2021-05-13 11:50:51...."
999996,4439990,"[1389, 22093, 7833, 17089, 1131, 4028, 17429, ...","[2020-08-22 11:39:59.191, 2020-08-22 12:33:15...."
999997,4439994,[25357],[2020-11-27 20:08:37.262]
999998,4439999,"[13056, 8433, 1299, 2173, 6007, 8502, 15853]","[2020-11-27 10:53:33.83, 2020-11-27 10:56:04.4..."


Test code zum überprüfen

In [ ]:
from datetime import datetime
datetime_list = [
    datetime(2009, 10, 12, 10, 10),
    datetime(2010, 10, 12, 10, 10),
    datetime(2010, 10, 12, 10, 10),
    datetime(2015, 2, 12, 10, 10), # future
    datetime(2016, 2, 12, 10, 10), # future
]
oldest = min(datetime_list)
youngest = max(datetime_list)

In [ ]:
youngest

datetime.datetime(2016, 2, 12, 10, 10)

In [ ]:
train_session_firstrow = train_session_sessionId_group_dataframe.iloc[0]

In [ ]:
highest_date = max(train_session_firstrow['date'])

In [ ]:
train_session_firstrow['date'].index(highest_date)

0

In [ ]:
len(itemDict)

23691

In [ ]:
item_one = itemDict[11662]

In [ ]:
item_two = itemDict[13226]

In [ ]:
from scipy.spatial import distance
test_dist = distance.cosine(item_one, item_two)

In [ ]:
test_dist

0.37239402523638765

In [ ]:
from scipy.spatial import minkowski_distance
test_dist2 = distance.euclidean(item_one, item_two)

In [ ]:
test_dist2

1734.109858111648

In [ ]:
from random import randint
for _ in range(10):
	value = randint(0, 0)
	print(value)

Distance Matrix erstellen 

Die Distance matrix zu erstellen dauert um den dreh 7 mins

In [ ]:
%%time

from scipy.spatial import distance_matrix
#distance_matrix(list(itemDict.values()), list(itemDict.values()))
distanceMatrix = pd.DataFrame(distance_matrix(list(itemDict.values()), list(itemDict.values())), columns=itemDict.keys(), index=itemDict.keys())

CPU times: user 6min 30s, sys: 16.4 s, total: 6min 47s
Wall time: 6min 48s


In [ ]:
distanceMatrix.head(10)

,2,3,4,7,8,9,10,11,13,14,...,28133,28134,28135,28137,28138,28139,28140,28141,28142,28143
2,0.000000,2856.469324,2716.219984,2054.486797,2999.781492,2853.184186,1798.609463,2593.646468,1603.611237,2049.745594,...,2257.172346,1572.787653,2562.943230,1656.697921,2864.588627,2806.914142,2353.051423,2669.690806,1891.854646,2007.062281
3,2856.469324,0.000000,1512.405369,2530.974318,999.138629,1556.612669,2913.213174,3237.284201,2543.863204,2792.046561,...,2831.140053,2887.068756,3064.945840,2620.915298,1394.763421,1959.580567,3046.024951,1854.307957,2605.599163,2405.748948
4,2716.219984,1512.405369,0.000000,2275.047912,1572.047073,1769.087618,2707.682958,3143.069042,2435.575497,2710.637563,...,2603.957373,2668.194146,2938.983668,2409.224979,1830.716526,1757.769041,3019.907283,1904.241056,2381.487140,2462.032494
7,2054.486797,2530.974318,2275.047912,0.000000,2622.524547,2704.300279,2023.990613,2653.086504,2045.459606,2302.934867,...,2101.796137,2108.003083,2625.239799,1906.807804,2533.165608,2364.115691,2554.551820,2548.680639,1346.505106,1936.898810
8,2999.781492,999.138629,1572.047073,2622.524547,0.000000,1574.396075,3006.037092,3152.311374,2746.583696,2920.077396,...,2901.503748,2926.345844,3169.277993,2729.684780,1461.127989,1988.214274,3226.501511,1947.209799,2648.501274,2315.756464
9,2853.184186,1556.612669,1769.087618,2704.300279,1574.396075,0.000000,2902.083734,3188.912667,2501.039984,2575.344055,...,2725.569849,2857.330747,3009.652139,2525.163757,1801.179613,1823.846211,3075.143411,1987.777402,2630.919231,2111.523384
10,1798.609463,2913.213174,2707.682958,2023.990613,3006.037092,2902.083734,0.000000,1979.266531,1837.513809,2422.616973,...,1781.358751,1942.370459,1872.774946,2088.097699,2994.353687,2715.705249,2689.100779,2998.367056,2191.732648,2294.674051
11,2593.646468,3237.284201,3143.069042,2653.086504,3152.311374,3188.912667,1979.266531,0.000000,2894.649720,2727.224047,...,2052.603956,2618.040680,2216.840545,2827.746453,3253.211951,3128.328787,3040.802032,3004.326380,2423.169412,2321.090907
13,1603.611237,2543.863204,2435.575497,2045.459606,2746.583696,2501.039984,1837.513809,2894.649720,0.000000,2287.936625,...,1681.021713,2040.005882,2152.474622,1269.670430,2484.854322,2460.021545,2439.571684,2584.046826,2204.236603,2232.667463
14,2049.745594,2792.046561,2710.637563,2302.934867,2920.077396,2575.344055,2422.616973,2727.224047,2287.936625,0.000000,...,2551.862065,2147.565599,2883.898923,2288.419760,2782.094355,2847.906248,2302.959835,2511.843148,2068.899466,1915.401263


In [ ]:
dist_row = distanceMatrix.loc[[2]]

In [ ]:
dist_row_dict = dist_row.to_dict(orient='records')

In [ ]:
dist_row_dict_sorted = dict(sorted(dist_row_dict[0].items(), key=lambda item: item[1]))

In [ ]:
dist_row_dict_sorted_top_100 = dict(list(dist_row_dict_sorted.items())[:100])

In [ ]:
dist_row_dict_sorted_top_100

In [ ]:
for key in dist_row_dict_sorted_top_100:
  print(key)


Help funktion 

nicht die idialste funtion da sie nur ein item betrachtet welches sich der user angeguckt hat 

In [ ]:
def createSessionDataFrame(sessionGroupDict):
  df_current = pd.DataFrame(columns=['session_id', 'item_id', 'rank'])

  i = 0

  for index, row in sessionGroupDict.iterrows():    

    if i == 50:
      break;

    sessionItemId = row['item_id'][0]

    dist_row = distanceMatrix.loc[[sessionItemId]]

    dist_row_dict = dist_row.to_dict(orient='records')

    dist_row_dict_sorted = dict(sorted(dist_row_dict[0].items(), key=lambda item: item[1]))

    dist_row_dict_sorted_top_100 = dict(list(dist_row_dict_sorted.items())[:100])

    itemRank = 1;

    for key in dist_row_dict_sorted_top_100:

      ser_i = pd.Series({'session_id':row['session_id'],
                    'item_id': key,
                    'rank':itemRank})
      itemRank += 1

      df_current = df_current.append(ser_i, ignore_index=True)

    i += 1
  
  return df_current

    

In [ ]:
dataFrame = createSessionDataFrame(train_session_sessionId_group_dataframe);

KeyboardInterrupt: ignored

In [ ]:
dataFrame

,session_id,item_id,rank
0,3,9655,1
1,3,7104,2
2,3,903,3
3,3,27603,4
4,3,143,5
...,...,...,...
4995,315,11046,96
4996,315,3966,97
4997,315,17487,98
4998,315,21174,99


In [ ]:
test_dings = dataFrame[(dataFrame['session_id'] == 77) & (dataFrame['item_id'] == 4243)]

In [ ]:
test_dings.empty

True

In [ ]:
not True 

False

Der algo soll nun erweitert werden so dass alle items betrachtet werden 

In [ ]:
def createSessionDataFrameWithAllItems(sessionGroupDict):
  df_current = pd.DataFrame(columns=['session_id', 'item_id', 'rank'])

  i = 0

  for index, row in sessionGroupDict.iterrows():    

    if i == 10:
      break;

    length_of_items = len(row["item_id"]);

    for j in range(1, 101):

      selectedItemId = randint(0, length_of_items - 1);

      sessionItemId = row['item_id'][selectedItemId]

      dist_row = distanceMatrix.loc[[sessionItemId]]

      dist_row_dict = dist_row.to_dict(orient='records')

      dist_row_dict_sorted = dict(sorted(dist_row_dict[0].items(), key=lambda item: item[1]))

      dist_row_dict_sorted_top_100 = dict(list(dist_row_dict_sorted.items()))

      for key in dist_row_dict_sorted_top_100:

        key_does_not_exists = df_current[(df_current['session_id'] == row['session_id']) & (df_current['item_id'] == key)].empty

        if not key_does_not_exists:
          continue
        
        ser_i = pd.Series({'session_id':row['session_id'],
                    'item_id': key,
                    'rank':j})

        df_current = df_current.append(ser_i, ignore_index=True)

        break

    i += 1

  return df_current

In [ ]:
dataFrame2 = createSessionDataFrameWithAllItems(train_session_sessionId_group_dataframe);
dataFrame2

In [ ]:
dataFrame2

,session_id,item_id,rank
0,3,9655,1
1,3,7104,2
2,3,903,3
3,3,27603,4
4,3,143,5
...,...,...,...
995,44,15403,96
996,44,8154,97
997,44,19537,98
998,44,12436,99


In [ ]:
test_dings = dataFrame[(dataFrame['session_id'] == 13) & (dataFrame['item_id'] == 18626)]

In [ ]:
test_dings

,session_id,item_id,rank


Ein weiterer ansatzt wäre die komplette evaluirung auf dem letzten item was angeschaut wurde 

In [ ]:
def createSessionDataFrameWithLastItemViewed(sessionGroupDict):
  df_current = pd.DataFrame(columns=['session_id', 'item_id', 'rank'])

  i = 0

  for index, row in sessionGroupDict.iterrows():    

    if i == 10:
      break;

    last_item_view_index = row['date'].index(max(row['date']))

    sessionItemId = row['item_id'][last_item_view_index]

    dist_row = distanceMatrix.loc[[sessionItemId]]

    dist_row_dict = dist_row.to_dict(orient='records')

    dist_row_dict_sorted = dict(sorted(dist_row_dict[0].items(), key=lambda item: item[1]))

    dist_row_dict_sorted_top_100 = dict(list(dist_row_dict_sorted.items())[:100])

    itemRank = 1;

    for key in dist_row_dict_sorted_top_100:

      ser_i = pd.Series({'session_id':row['session_id'],
                    'item_id': key,
                    'rank':itemRank})
      itemRank += 1

      df_current = df_current.append(ser_i, ignore_index=True)

    i += 1
  
  return df_current

In [ ]:
dataFrame3 = createSessionDataFrameWithLastItemViewed(train_session_sessionId_group_dataframe);
dataFrame3

,session_id,item_id,rank
0,3,9655,1
1,3,7104,2
2,3,903,3
3,3,27603,4
4,3,143,5
...,...,...,...
995,44,13089,96
996,44,702,97
997,44,3863,98
998,44,10045,99


In [ ]:
dataFrame3[(dataFrame3['session_id'] == 42) & (dataFrame3['item_id'] == 11784)]

,session_id,item_id,rank
